<a href="https://colab.research.google.com/github/jfauv/Advanced-CV-TF-Coursera/blob/master/C3_W2_Lab_1_Simple_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Object Detection in Tensorflow

This lab will walk you through how to use object detection models available in [Tensorflow Hub](https://www.tensorflow.org/hub). In the following sections, you will:

* explore the Tensorflow Hub for object detection models
* load the models in your workspace
* preprocess an image for inference
* run inference on the models and inspect the output

Let's get started!

## Imports

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

### Download the model from Tensorflow Hub

Tensorflow Hub is a repository of trained machine learning models which you can reuse in your own projects.
- You can see the domains covered [here](https://tfhub.dev/) and its subcategories.
- For this lab, you will want to look at the [image object detection subcategory](https://tfhub.dev/s?module-type=image-object-detection).
- You can select a model to see more information about it and copy the URL so you can download it to your workspace.
- We selected a [inception resnet version 2](https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1)
- You can also modify this following cell to choose the other model that we selected, [ssd mobilenet version 2](https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2)

In [2]:
# you can switch the commented lines here to pick the other model

# inception resnet version 2
#module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# You can choose ssd mobilenet version 2 instead and compare the results
module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

#### Load the model

Next, you'll load the model specified by the `module_handle`.
- This will take a few minutes to load the model.

In [3]:
model = hub.load(module_handle)

#### Choose the default signature

Some models in the Tensorflow hub can be used for different tasks. So each model's documentation should show what *signature* to use when running the model.
- If you want to see if a model has more than one signature then you can do something like `print(hub.load(module_handle).signatures.keys())`. In your case, the models you will be using only have the `default` signature so you don't have to worry about other types.

In [4]:
hub.load(module_handle).signatures.keys()

KeysView(_SignatureMap({'default': <ConcreteFunction pruned(images) at 0x7E5DBF352260>}))

Please choose the 'default' signature for your object detector.
- For object detection models, its 'default' signature will accept a batch of image tensors and output a dictionary describing the objects detected, which is what you'll want here.

In [5]:
detector = model.signatures["default"]

### download_and_resize_image

This function downloads an image specified by a given "url", pre-processes it, and then saves it to disk.

In [6]:
def download_and_resize_image(url, new_width=256, new_height=256):
  """
  Fetches an image online, resizes it and saves it locally.

  Args:
    url (string) -- link to the image
    new_width (int) -- size iin pixels used for resizing the width of the image
    new_height (int) -- size in pixels used for resizing the length of the image

  Returns:
    (string) -- path to saved image
  """

  # create a temporary file ending with ".jpg"
  _, filename = tempfile.mkstemp(suffix=".jpg")

  # opens the given URL
  response = urlopen(url)

  # reads the image fetched from the URL
  image_data = response.read()

  # puts the image data in memory buffer
  image_data = BytesIO(image_data)

  # opens the image
  pil_image = Image.open(image_data)

  # resizes the image. will crop if aspect ratio is different.
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)

  # converts to the RGB colorspace
  pil_image_rgb = pil_image.convert("RGB")

  # saves the image to the temporary file created earlier
  pil_image_rgb.save(filename, format="JPEG", quality=90)

  print("Image downloaded to %s." % filename)

  return filename

### Download and preprocess an image

Now, using `download_and_resize_image` you can get a sample image online and save it locally.
- We've provided a URL for you, but feel free to choose another image to run through the object detector.
- You can use the original width and height of the image but feel free to modify it and see what results you get.

In [7]:
# You can choose a different URL that points to an image of your choice
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# download the image and use the original height and width
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

Image downloaded to /tmp/tmp928pdo7n.jpg.


<ipython-input-6-3accc599519f>:30: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)


### run_detector

This function will take in the object detection model `detector` and the path to a sample image, then use this model to detect objects and display its predicted class categories and detection boxes.
- run_detector uses `load_image` to convert the image into a tensor.

In [8]:
def load_img(path):
  """
  Loads a JPEG image and converts it to a tensor.

  Args:
    path (string) -- path to a locally saved JPEG image

  Returns:
    (tensor) -- an image tensor
  """

  # read the file
  img = tf.io.read_file(path)

  # convert to a tensor
  img = tf.image.decode_jpeg(img, channels=3)

  return img

def run_detector(detector, path):
  """
  Runs inference on a local files using an object detection model.

  Args:
    detector (model) -- an object detection model loaded from TF hub
    path (string) -- path to an image saved locally
  """

  # load an image tensor from a local file path
  img = load_img(path)

  # add a batch dimension in front of the tensor
  converted_img = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]

  # run inference using the model
  result = detector(converted_img)

  # save the results in a dictionary
  result = {key: value.numpy() for key, value in result.items()}

  # print results
  print("Found %d objects." % len(result["detection_scores"]))

  print(result["detection_scores"])
  print(result["detection_class_entities"])
  print(result["detection_boxes"])

### Run inference on the image

You can run your detector by calling the `run_detector` function. This will print the number of objects found followed by three lists:

* The detection scores of each object found (i.e. how confident the model is),
* The classes of each object found,
* The bounding boxes of each object

You will see how to overlay this information on the original image in the next sections and in this week's assignment!

In [9]:
# runs the object detection model and prints information about the objects found
run_detector(detector, downloaded_image_path)

Found 100 objects.
[0.436388   0.3488732  0.24501613 0.23388161 0.22632208 0.2156073
 0.20603243 0.20521504 0.20307639 0.19840115 0.18880866 0.18106377
 0.18082245 0.17998277 0.17989479 0.17721444 0.17477798 0.17303872
 0.17053543 0.16971526 0.16919276 0.16825813 0.1642754  0.16199107
 0.16110268 0.16060077 0.16035771 0.1596943  0.1577436  0.15739347
 0.15714587 0.15653816 0.15279417 0.1499805  0.14963487 0.14810573
 0.14709249 0.1470065  0.14588015 0.1457028  0.14534342 0.14526632
 0.14403893 0.14400737 0.14383514 0.14284107 0.14267781 0.14242578
 0.14031248 0.13963945 0.13949175 0.13740636 0.13706668 0.13699545
 0.13636456 0.13557202 0.13488966 0.13072284 0.1301368  0.12861584
 0.12749374 0.12689273 0.1265813  0.12564696 0.12514336 0.1230045
 0.12287524 0.12273064 0.12224491 0.1222197  0.12181569 0.12164262
 0.1208571  0.12058606 0.11993121 0.11953843 0.11838248 0.11800458
 0.11747291 0.11726157 0.11694559 0.11667921 0.11636377 0.11611395
 0.11582799 0.11541581 0.11513458 0.11508507 